In [92]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

## Importing the functions to calculate the metrices for splitting

In [2]:
def gini_impurity(labels):
    #when the set is empty it is pure
    if labels.size == 0:
        return 0
    counts = np.unique(labels,return_counts=True)[1]
    fractions = counts / float(len(labels))
    return 1 - np.sum(fractions**2)
    

In [3]:
def entropy(labels):
    # When the set is empty, it is also pure
    if labels.size == 0:
        return 0
    counts = np.unique(labels, return_counts=True)[1]
    fractions = counts / float(len(labels))
    return - np.sum(fractions * np.log2(fractions))

In [4]:
criterion_function = {'gini': gini_impurity, 'entropy': entropy}
def weighted_impurity(groups, criterion='gini'):
    """ Calculate weighted impurity of children after a split
    Args:
        groups (list of children, and a child consists a list of class labels)
        criterion (metric to measure the quality of a split, 'gini' for Gini Impurity or 'entropy' for Information Gain)
    Returns:
        float, weighted impurity
    """
    total = sum(len(group) for group in groups)
    weighted_sum = 0.0
    for group in groups:
        weighted_sum += len(group) / float(total) * criterion_function[criterion](group)
    return weighted_sum


In [79]:
def split_node(X,y,index,value):
    """
    Args - 
      X - nparray(features)
      Y - ndarray(labels)
      index - index of the col used to split
      value - value of feature in the feature column used for splitting
      
    Returns - 
    left,right - List of list, each list is in form of [X,Y], representing
                  left and right child after splitting
    """
    X_index = X[:,index]
    if X[0,index].dtype.kind in ['i','f']:
        mask = X_index >= value
    else:
        mask = X_index == value   
    #split into left and right child
    left = [np.array(X[~mask]),np.array(y[~mask])]
    right = [np.array(X[mask]),np.array(y[mask])]
    return left,right

In [15]:
def get_best_split(X,y,criterion = 'gini'):
    """
    ARGS - 
      X - ndarray(features)
      Y - ndarray(labels)
      criterion - 'gini' or 'entropy'
    Returns - 
      dict {'index':___,'value':_____,'children':_____}
           where - dict['children'][0] = left_child
                   dict['children'][1] = right_child]
    """
    best_index,best_value,best_score,children = None,None,1,None
    for index in range(X_train.shape[1]):
        for value in np.unique(X[:,index]):
            groups = split_node(X,y,index,value)
            score = weighted_impurity([groups[0][1],groups[1][1]])
            if score < best_score:
                best_index,best_value,best_score,children = index,value,score,groups
    return {'index':best_index,'value':best_value,'children':children}            
            
        

In [16]:
def get_leaf(labels):
    #the value of the leaf is majority of the labels
    return np.bincount(labels).argmax()

In [84]:
def split(node,max_depth,min_size,depth,criterion):
    """
    AGRS - 
       node - (dict with children info)
       max_depth - int - max depth of the tree
       min_size - int - min size required for splitting
       depth - int - current depth
       criterion - 'gini' or 'entropy'
    """
    left,right = node['children']
    del(node['children'])
    
#    #for debugging
#     print('Max Depth - {}, Min size - {}, Current deplt - {}'.format(max_depth,min_size,depth))
#     print('type of node - ',type(node))
#     print('type of child - ',type(left))
#     print(left[1].size)

    #first_stopping condition
    if left[1].size == 0:
        node['right'] = get_leaf(right[1])
        return
    
    if right[1].size == 0:
        node['left'] = get_leaf(left[1])
        return 
    
    #second stoppoing condition
    #check if the current depth is greater than max depth
    if depth >= max_depth:
        node['left'],node['right'] = get_leaf(left[1]),get_leaf(right[1])
        return
    
    #left child
    
    #stopping condition
    if left[1].size <= min_size:
        node['left'] = get_leaf(left[1])
        
    else:
        result = get_best_split(left[0],left[1],criterion)
        result_left,result_right = result['children']
        if result_left[1].size == 0:
            node['left'] = get_leaf(result_right[1])
        elif result_right[1].size == 0:
            node['left'] = get_leaf(result_left[1])
        else:
            node['left'] = result
            split(node['left'],max_depth,min_size,depth+1,criterion)
            
    #right node
    
    #stopping criterion
    if right[1].size <= min_size:
        node['right'] = get_leaf(right[1])
        
    else:
        result = get_best_split(right[0],right[1],criterion)
        result_left,result_right = result['children']
        if result_left[1].size == 0:
            node['right'] = get_leaf(result_right[1])
        elif result_right[1].size == 1:
            node['right'] = get_leaf(result_left[1])
        else:
            node['right'] = result
            split(node['right'],max_depth,min_size,depth+1,criterion)
            
        
        

In [81]:
CONDITION = {'numerical': {'yes': '>=', 'no': '<'},
             'categorical': {'yes': 'is', 'no': 'is not'}}
def visualize_tree(node, depth=0):
    if isinstance(node, dict):
        if node['value'].dtype.kind in ['i', 'f']:
            condition = CONDITION['numerical']
        else:
            condition = CONDITION['categorical']
        print('{}|- X{} {} {}'.format(depth * '  ', node['index'] + 1, condition['no'], node['value']))
        if 'left' in node:
            visualize_tree(node['left'], depth + 1)
        print('{}|- X{} {} {}'.format(depth * '  ', node['index'] + 1, condition['yes'], node['value']))
        if 'right' in node:
            visualize_tree(node['right'], depth + 1)
    else:
        print('{}[{}]'.format(depth * '  ', node))


In [86]:
def train_tree(X_train,y_train,max_depth,min_size,criterion):
    """
    Contructs a tree
    Agrs - 
      X_train,y_train - list - (features,labels)
      max_depth - max_depth of the decision tree
      min_size - min_size of the tree
      criterion - 'gini' or 'entropy'
    Returns - 
      Root - Root node of the tree
      """
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    root = get_best_split(X_train,y_train,criterion)
    split(root,max_depth,min_size,1,criterion)
    return root

In [82]:

X_train = np.array([['tech', 'professional'],
           ['fashion', 'student'],
           ['fashion', 'professional'],
           ['sports', 'student'],
           ['tech', 'student'],
           ['tech', 'retired'],
           ['sports', 'professional']])

y_train = np.array([1,
           0,
           0,
           0,
           1,
           0,
           1])


In [88]:
tree = train_tree(X_train, y_train, 2, 2,'gini')
visualize_tree(tree)

|- X1 is not fashion
  |- X2 is not professional
    [0]
  |- X2 is professional
    [1]
|- X1 is fashion
  [0]


In [89]:

X_train_n = [[6, 7],
           [2, 4],
           [7, 2],
           [3, 6],
           [4, 7],
           [5, 2],
           [1, 6],
           [2, 0],
           [6, 3],
           [4, 1]]

y_train_n = [0,
           0,
           0,
           0,
           0,
           1,
           1,
           1,
           1,
           1]

In [91]:
tree = train_tree(X_train_n, y_train_n, 2, 2,'gini')
visualize_tree(tree)

|- X2 < 4
  |- X1 < 7
    [1]
  |- X1 >= 7
    [0]
|- X2 >= 4
  |- X1 < 2
    [1]
  |- X1 >= 2
    [0]


## Using built in tree from sklearn

In [93]:
 tree = DecisionTreeClassifier(criterion='gini',max_depth=2,min_samples_split=2)
tree.fit(X_train_n,y_train_n)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [94]:
tree.predict(X_train_n)

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

In [96]:
from sklearn.tree import export_graphviz
export_graphviz(tree, out_file='tree.dot', feature_names=['X1', 'X2'], impurity=False, filled=True, class_names=['0', '1'])